In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import warnings
import os
from tqdm import tqdm
import numpy as np
from scipy.spatial import ConvexHull
warnings.filterwarnings('ignore')
from shapely.geometry import Point, Polygon
from shapely.prepared import prep
from rtree import index

In [2]:
adata = sc.read_h5ad('./raw_counts_mouse3_sagittal.h5ad')
print(adata.shape)
adata.obs

(2234445, 1147)


,fov,volume,center_x,center_y,sample_id,slice_id,fov_x,fov_y
149186845364815977486646456377093859505,0,204.802526,-5403.409202,5618.734442,sa1_sample17,sa1_slice23,-5603.7,5523.20
165947034672980197818204734225096179101,0,144.322419,-5392.125202,5564.809944,sa1_sample17,sa1_slice23,-5603.7,5523.20
198904341065180396762707397604803217407,0,286.491929,-5386.054628,5607.613192,sa1_sample17,sa1_slice23,-5603.7,5523.20
228959056670638687442667322030441690553,0,551.802639,-5391.040202,5555.858694,sa1_sample17,sa1_slice23,-5603.7,5523.20
252199681526991424029643077826220097990,0,330.835990,-5402.107202,5600.126693,sa1_sample17,sa1_slice23,-5603.7,5523.20
...,...,...,...,...,...,...,...,...
90449937054422809676894946394739204338,1687,436.382040,-5757.270309,-3175.154446,sa1_sample9,sa1_slice9,-5950.8,-3366.94
95504860602157630895026940886091487621,1687,681.661293,-5827.520808,-3320.233443,sa1_sample9,sa1_slice9,-5950.8,-3366.94
298741194638354463703125568813707946720,1688,507.491782,-5781.577309,-3355.915946,sa1_sample9,sa1_slice9,-5950.8,-3566.94
338908751809376723590481124346144852638,1688,756.083873,-5768.442809,-3368.559946,sa1_sample9,sa1_slice9,-5950.8,-3566.94


In [3]:
path = '/mnt/Data18Td/Data/haichao/merfish_raw_data_zxw3/'
decode_files_name = os.listdir(path + 'decode/')
decode_files_name

['spots_220713_wb3_sa1_B_13_20_5z18R_merfish5.csv',
 'spots_220706_wb3_sa1_B_12_21_5z18R_merfish5.csv',
 'spots_220717_wb3_sa1_B_9_5z18R_merfish5.csv',
 'spots_220609_wb3_sa1_1_5z18R_merfish5.csv',
 'spots_220627_wb3_sa1_6_5z18R_merfish6.csv',
 'spots_220613_wb3_sa1_2_5z18R_merfish5.csv',
 'spots_220506_sa_10_merfish4_adaptor_5z18r.csv',
 'spots_220606_wb3_sa1_3_5z18R_merfish6.csv',
 'spots_220710_wb3_sa1_B_7_5z18R_merfish5.csv',
 'spots_220424_sa_16_17_merfish4_adaptor_5z18r.csv',
 'spots_220616_wb3_sa1_B_11_22_5z18R_merfish6.csv',
 'spots_220421_sa_23_24_25_merfish4_adaptor_5z18r.csv',
 'spots_220612_sa_15_18B_merfish4_adaptor.csv',
 'spots_220608_sa_4_merfish4_adaptor.csv',
 'spots_220620_sa_14_19B_merfish4_adaptor.csv']

In [4]:
def create_cell_index(cells):
    idx = index.Index()
    for i, cell in enumerate(cells):
        idx.insert(i, Polygon(cell).bounds)
    return idx

def point_in_polygon(point, polygon):
    return Point(point).within(Polygon(polygon))

def assign_rna_to_cells(rna_points, cells, cell_name):
    cell_index = create_cell_index(cells)
    results = []
    
    for rna_point in tqdm(rna_points):
        potential_cells = list(cell_index.intersection(Point(rna_point).bounds))
        cell_assigned = False
        for cell_id in potential_cells:
            if point_in_polygon(rna_point, cells[cell_id]):
                results.append({
                    'cell_id': cell_name[cell_id],
                    'rna_x': rna_point[0],
                    'rna_y': rna_point[1]
                })
                cell_assigned = True
                break
        
        if not cell_assigned:
            results.append({
                'cell_id': -1,
                'rna_x': rna_point[0],
                'rna_y': rna_point[1]
            })
    
    return pd.DataFrame(results)

In [5]:
for f in decode_files_name:
    print(f)
    decode = pd.read_csv(path + 'decode/' + f)
    cellpose = pd.read_csv(path + 'cell_boundary/' + f[6:], index_col=0)
    cellpose = cellpose[cellpose.index.isin(adata.obs.index)]

    for z in decode['global_z'].unique():
        z = int(z)
        boundary_x_col = f'boundaryX_z{z}'
        boundary_y_col = f'boundaryY_z{z}'
        cellpose_z = cellpose[[boundary_x_col, boundary_y_col]]
        cellpose_z = cellpose_z.dropna()
        cellpose_z = cellpose_z[cellpose_z.index.isin(adata.obs.index)]  # choose the cell which in the adata
        cells = []
        for idx in cellpose_z.index:
            x1 = np.array(cellpose.loc[idx, boundary_x_col].split(', '), dtype=float)
            y1 = np.array(cellpose.loc[idx, boundary_y_col].split(', '), dtype=float)
            cells.append(list(zip(x1, y1)))

        dec_z = decode[decode['global_z'] == z]
        rna_points = dec_z[['global_x', 'global_y']].values
        result_df = assign_rna_to_cells(rna_points, cells, cellpose_z.index)
        result_df.to_csv(f'./RNA_assign_result/z{z}_{f}')

spots_220424_sa_16_17_merfish4_adaptor_5z18r.csv


100%|██████████| 27674708/27674708 [2:02:26<00:00, 3766.94it/s]  


spots_220616_wb3_sa1_B_11_22_5z18R_merfish6.csv


100%|██████████| 19425171/19425171 [1:17:54<00:00, 4155.19it/s]  


spots_220421_sa_23_24_25_merfish4_adaptor_5z18r.csv


100%|██████████| 11258951/11258951 [58:14<00:00, 3221.80it/s] 
